In [17]:
import pandas as pd
import os
import gzip
import json
import zlib
import numpy as np
import sys


import ast


import numpy as np
import importlib


import sys
import importlib


sys.path.append('data/data_utils')
import data_utils
importlib.reload(data_utils)
from data_utils import *

In [16]:
run_id = 513
file_path= 'f/checkpoint/maui_sft/winnieyangwn/amaia_dumps/{run_id}/trajectories/mle_bench_bashmle_bench_checkpoint_maui_sft_shared_kniu_datasets_mlebench_full_jsonl/mle_bench_bashmle_bench_checkpoint_maui_sft_shared_kniu_datasets_mlebench_full_jsonl.jsonl'


In [18]:
df_trj = pd.read_json(file_path, lines=True)


/tmp/ipykernel_767782/3726597042.py:1: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df_trj = pd.read_json(file_path, lines=True)


ValueError: Unexpected character found when decoding 'false'

In [19]:

# Flatten the dataframe
df_flat = flatten_dataframe(df_trj)

print(f"Shape: {df_flat.shape}")
print(f"Columns: {df_flat.columns.tolist()}")
df_flat.head()

Shape: (2526, 9)
Columns: ['task_name', 'task_description', 'code', 'percentile', 'valid_submission', 'eval_error_message', 'eval_duration', 'rollout_duration', 'rollout']


,task_name,task_description,code,percentile,valid_submission,eval_error_message,eval_duration,rollout_duration,rollout
0,aerial-cactus-identification,# Overview\n\n## Overview\n\n### Description\n...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.0,False,Execution returned non-zero exit code,0.0,186.208566,"[{'turn_id': 0, 'action': '', 'observation': '..."
1,aerial-cactus-identification,# Overview\n\n## Overview\n\n### Description\n...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.0,False,Execution returned non-zero exit code,0.0,188.979062,"[{'turn_id': 0, 'action': '', 'observation': '..."
2,denoising-dirty-documents,# Overview\n\n## Description\n\n[Optical Chara...,import os\nimport sys\nimport math\nimport csv...,0.0,False,Execution returned non-zero exit code,0.0,194.721609,"[{'turn_id': 0, 'action': '', 'observation': '..."
3,denoising-dirty-documents,# Overview\n\n## Description\n\n[Optical Chara...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.0,False,Execution returned non-zero exit code,0.0,197.420535,"[{'turn_id': 0, 'action': '', 'observation': '..."
4,denoising-dirty-documents,# Overview\n\n## Description\n\n[Optical Chara...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.0,False,Execution returned non-zero exit code,0.0,198.732197,"[{'turn_id': 0, 'action': '', 'observation': '..."


In [ ]:
df_trj.iloc[0]["rollouts"][0]["metrics"]["rollout/duration"]

186.20856591500342

In [ ]:
df_trj.iloc[0]["rollouts"][0]["traj"]["transitions"][-1]["info"]["pred_solution"]

'#!/usr/bin/env python3\nimport os\nimport sys\nimport time\nimport json\nimport math\nimport random\nimport argparse\nfrom dataclasses import dataclass\n\n# Attempt to import required packages; install if missing\n\ndef ensure_imports():\n    import importlib\n    need = []\n    for pkg in [\n        (\'torch\', \'torch\'),\n        (\'torchvision\', \'torchvision\'),\n        (\'sklearn\', \'sklearn\'),\n        (\'pandas\', \'pandas\'),\n        (\'numpy\', \'numpy\'),\n        (\'PIL\', \'PIL\'),\n    ]:\n        try:\n            importlib.import_module(pkg[0])\n        except Exception:\n            need.append(pkg[1])\n    if need:\n        print(f"[setup] Installing missing packages: {need}")\n        import subprocess\n        subprocess.check_call([sys.executable, \'-m\', \'pip\', \'install\', \'--no-input\'] + need)\n\nensure_imports()\n\nimport numpy as np\nimport pandas as pd\nfrom PIL import Image\nimport torch\nimport torch.nn as nn\nimport torch.nn.functional as F\nfrom

In [ ]:
def build_rollout(df_trj, rollout_id: int) -> list[dict]:
    """
    Build a rollout as a list of turn dicts for a given rollout_id.
    
    Args:
        df_trj: DataFrame containing trajectory data
        rollout_id: Index of the rollout in the DataFrame
        
    Returns:
        List of dicts, each with "action" and "observation" fields
    """
    transitions = df_trj.iloc[rollout_id]["rollouts"][0]["traj"]["transitions"]
    
    rollout = []
    for t, turn in enumerate(transitions):
        rollout.append({
            "turn_id": t,
            "action": turn["action_str"],
            "observation": turn["observation_str"]
        })
    
    return rollout



In [13]:
def flatten_dataframe(df_trj) -> pd.DataFrame:
    """
    Flatten the trajectory DataFrame into a simplified pandas DataFrame.
    
    Returns:
        DataFrame with task_name, task_description, code, percentile, valid_submission,
        eval_error_message, eval_duration, rollout_duration, and rollout columns.
    """
    rows = []
    
    for idx in range(len(df_trj)):
        row = df_trj.iloc[idx]
        rollout_data = row["rollouts"][0]
        transitions = rollout_data["traj"]["transitions"]
        last_outcomes = transitions[-1]["outcomes"]
        info = transitions[-1]["info"]
        
        rows.append({
            "task_name": rollout_data["start_args"]["instance_id"],
            "task_description": rollout_data["start_args"]["task_description"],
            "code": info.get("pred_solution"),
            "percentile": last_outcomes.get("percentile"),
            "valid_submission": last_outcomes.get("valid_submission"),
            "eval_error_message": last_outcomes.get("eval_error_message"),
            "eval_duration": last_outcomes.get("gpu_execution_duration"),
            "rollout_duration": rollout_data["metrics"].get("rollout/duration"),
            "rollout": build_rollout(df_trj, idx)
        })
    
    return pd.DataFrame(rows)


# Flatten the dataframe
df_flat = flatten_dataframe(df_trj)

print(f"Shape: {df_flat.shape}")
print(f"Columns: {df_flat.columns.tolist()}")
df_flat.head()

Shape: (2526, 9)
Columns: ['task_name', 'task_description', 'code', 'percentile', 'valid_submission', 'eval_error_message', 'eval_duration', 'rollout_duration', 'rollout']


,task_name,task_description,code,percentile,valid_submission,eval_error_message,eval_duration,rollout_duration,rollout
0,aerial-cactus-identification,# Overview\n\n## Overview\n\n### Description\n...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.0,False,Execution returned non-zero exit code,0.0,186.208566,"[{'turn_id': 0, 'action': '', 'observation': '..."
1,aerial-cactus-identification,# Overview\n\n## Overview\n\n### Description\n...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.0,False,Execution returned non-zero exit code,0.0,188.979062,"[{'turn_id': 0, 'action': '', 'observation': '..."
2,denoising-dirty-documents,# Overview\n\n## Description\n\n[Optical Chara...,import os\nimport sys\nimport math\nimport csv...,0.0,False,Execution returned non-zero exit code,0.0,194.721609,"[{'turn_id': 0, 'action': '', 'observation': '..."
3,denoising-dirty-documents,# Overview\n\n## Description\n\n[Optical Chara...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.0,False,Execution returned non-zero exit code,0.0,197.420535,"[{'turn_id': 0, 'action': '', 'observation': '..."
4,denoising-dirty-documents,# Overview\n\n## Description\n\n[Optical Chara...,#!/usr/bin/env python3\nimport os\nimport sys\...,0.0,False,Execution returned non-zero exit code,0.0,198.732197,"[{'turn_id': 0, 'action': '', 'observation': '..."


In [14]:
save_path = f'/checkpoint/maui_sft/winnieyangwn/amaia_dumps/{run_id}/trajectories/{run_id}_metadata.jsonl'

# Save df_flat to jsonl
df_flat.to_json(save_path, orient='records', lines=True)
print(f"Saved {len(df_flat)} rows to {save_path}")

Saved 2526 rows to /checkpoint/maui_sft/winnieyangwn/amaia_dumps/513/trajectories/513_metadata.jsonl


# Data Structure Description

The context is a list of dictionaries, where each dictionary represents one MLE Bench rollout (an agent's attempt to solve a machine learning task). There are 4,800 rollouts in total.

Each rollout dictionary has the following fields:

1. **"task_name"** (str): The unique identifier for the ML task (e.g., "detecting-insults-in-social-commentary")

2. **"task_description"** (str): Full markdown description of the ML task including overview, evaluation criteria, and submission format

3. **"code"** (str | None): The final Python code solution submitted by the agent

4. **"percentile"** (float | None): Performance percentile (0-100) achieved by the submission. Higher is better. 100 means top performer.

5. **"valid_submission"** (bool | None): Whether the agent produced a valid submission file

6. **"eval_error_message"** (str | None): Evaluation result message - contains success info or error details

7. **"eval_duration"** (float | None): GPU execution time in seconds for evaluation

8. **"rollout_duration"** (float | None): Total time in seconds for the entire rollout

9. **"rollout"** (list[dict]): The full conversation trajectory as a list of turns. Each turn has:
   - "turn_id" (int): Turn number starting from 0
   - "action" (str): The agent's action/response (typically bash commands in XML tags)
   - "observation" (str): The environment's response/prompt to the agent

## Example access patterns:
```python
# Get task name
context[0]["task_name"]

# Get percentile
context[0]["percentile"]

# Get number of turns
len(context[0]["rollout"])

# Get first action
context[0]["rollout"][0]["action"]

# Filter successful submissions
[r for r in context if r["valid_submission"] == True]

# Filter by percentile
[r for r in context if r["percentile"] and r["percentile"] >= 50]
```